In [13]:
import json
import pandas as pd
import numpy as np
import statistics

# loading XGBoost
with open("outputs/output xgboost.json", 'r') as f:
  data = json.load(f)

# sort dataset by normalization
df_XGBoost = pd.json_normalize(data, record_path =['measurements'])

# save minimum percentage to be dropped in extra variable
df_XGBoost["drop_columns"] = df_XGBoost["best_params.drop_columns__minimum_percentage_to_be_dropped"]
 

In [14]:
# calculate main values of interest
df_XGBoost["auc_std"] = df_XGBoost.apply(lambda row: np.std(row["auc"]), axis =1)
df_XGBoost["pre_mean"] = df_XGBoost.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
df_XGBoost["rec_mean"] = df_XGBoost.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
df_XGBoost["f1_std"] = df_XGBoost.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
df_XGBoost["f1_mean"] = df_XGBoost.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
df_XGBoost["type2"] = df_XGBoost.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
# Confidence Intervalls
df_XGBoost["auc_confl"] = df_XGBoost["auc_mean"]- 1.96 * (df_XGBoost["auc_std"] / np.sqrt(10))
df_XGBoost["auc_confu"] = df_XGBoost["auc_mean"]+ 1.96 * (df_XGBoost["auc_std"] / np.sqrt(10))
df_XGBoost["f1_confl"] = df_XGBoost["f1_mean"]- 1.96 * (df_XGBoost["f1_std"] / np.sqrt(10))
df_XGBoost["f1_confu"] = df_XGBoost["f1_mean"]+ 1.96 * (df_XGBoost["f1_std"] / np.sqrt(10))
# get a look of the final dataset
df_XGBoost.head()

,scaler,estimator,imputer,sampler,X_shape,one_hot_encoded_features,auc_mean,execution_time_in_seconds,auc,parameters.impute__strategy,...,auc_std,pre_mean,rec_mean,f1_std,f1_mean,type2,auc_confl,auc_confu,f1_confl,f1_confu
0,MaxAbsScaler,XGBClassifier,SimpleImputer,str,"[899, 43]","[cp, restecg, slope, ca, restwm]",75.027738,122.561587,"[0.714534594325535, 0.7675460428073669, 0.8188...","[mean, median, most_frequent]",...,0.076513,0.768763,0.753995,0.086839,0.744731,108,74.980315,75.075161,0.690908,0.798555
1,MaxAbsScaler,XGBClassifier,SimpleImputer,RandomOverSampler,"[899, 43]","[cp, restecg, slope, ca, restwm]",75.438477,123.505258,"[0.7349427575908413, 0.7797411647585863, 0.771...","[mean, median, most_frequent]",...,0.070096,0.775624,0.757341,0.080040,0.748220,110,75.395031,75.481923,0.698611,0.797830
2,MaxAbsScaler,XGBClassifier,SimpleImputer,RandomUnderSampler,"[899, 43]","[cp, restecg, slope, ca, restwm]",75.162469,107.387465,"[0.7553509208561474, 0.7899452463912394, 0.794...","[mean, median, most_frequent]",...,0.081176,0.765268,0.753970,0.091534,0.745038,114,75.112155,75.212783,0.688304,0.801771
3,MinMaxScaler,XGBClassifier,SimpleImputer,str,"[899, 43]","[cp, restecg, slope, ca, restwm]",75.050697,116.305835,"[0.714534594325535, 0.7675460428073669, 0.8188...","[mean, median, most_frequent]",...,0.075209,0.768886,0.753995,0.085831,0.744858,109,75.004082,75.097312,0.691660,0.798057
4,MinMaxScaler,XGBClassifier,SimpleImputer,RandomOverSampler,"[899, 43]","[cp, restecg, slope, ca, restwm]",75.438477,122.199511,"[0.7349427575908413, 0.7797411647585863, 0.771...","[mean, median, most_frequent]",...,0.070096,0.775624,0.757341,0.080040,0.748220,110,75.395031,75.481923,0.698611,0.797830


In [15]:
# only display variables important for the model evaluation
df_XGBoost = df_XGBoost.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
df_XGBoost[["auc_mean", "auc_std", "type2", "drop_columns"]].head(20)

,auc_mean,auc_std,type2,drop_columns
7,76.971665,0.074685,96,100
8,76.540518,0.086870,109,75
9,75.475697,0.080552,106,100
1,75.438477,0.070096,110,100
4,75.438477,0.070096,110,100
10,75.438477,0.070096,110,100
13,75.438477,0.070096,110,100
19,75.438477,0.070096,110,100
15,75.402738,0.075977,108,100
18,75.352738,0.076452,106,100


In [16]:
# best model
XGBoost = df_XGBoost.loc[8]
XGBoost[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

scaler                  Normalizer
estimator            XGBClassifier
imputer              SimpleImputer
sampler         RandomUnderSampler
auc_mean                 76.540518
auc_confl                76.486675
auc_confu                 76.59436
type2                          109
f1_mean                   0.759294
f1_confl                  0.700291
f1_confu                  0.818296
drop_columns                    75
pre_mean                  0.778677
rec_mean                  0.767316
Name: 8, dtype: object

In [17]:
# loading KNeighbors 
with open("outputs/output KNN.json", 'r') as f:
  data_KNeighbors = json.load(f)

df_KNeighbors = pd.json_normalize(data_KNeighbors, record_path =['measurements'])
df_KNeighbors["drop_columns"] = df_KNeighbors["best_params.drop_columns__minimum_percentage_to_be_dropped"]
df_KNeighbors["auc_std"] = df_KNeighbors.apply(lambda row: np.std(row["auc"]), axis =1)
df_KNeighbors["pre_mean"] = df_KNeighbors.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
df_KNeighbors["rec_mean"] = df_KNeighbors.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
df_KNeighbors["f1_std"] = df_KNeighbors.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
df_KNeighbors["f1_mean"] = df_KNeighbors.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
df_KNeighbors["type2"] = df_KNeighbors.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
df_KNeighbors["auc_confl"] = df_KNeighbors["auc_mean"]- 1.96 * (df_KNeighbors["auc_std"] / np.sqrt(10))
df_KNeighbors["auc_confu"] = df_KNeighbors["auc_mean"]+ 1.96 * (df_KNeighbors["auc_std"] / np.sqrt(10))
df_KNeighbors["f1_confl"] = df_KNeighbors["f1_mean"]- 1.96 * (df_KNeighbors["f1_std"] / np.sqrt(10))
df_KNeighbors["f1_confu"] = df_KNeighbors["f1_mean"]+ 1.96 * (df_KNeighbors["f1_std"] / np.sqrt(10))
df_KNeighbors.head()
df_KNeighbors = df_KNeighbors.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
df_KNeighbors[["auc_mean", "auc_std", "type2", "drop_columns"]].head(20)

,auc_mean,auc_std,type2,drop_columns
15,80.398731,0.097697,83,100
17,79.798258,0.090229,100,100
16,79.383997,0.092650,109,100
5,78.938589,0.063572,127,100
4,78.811013,0.065513,121,100
11,78.790070,0.090879,105,100
2,78.481906,0.065569,123,100
9,78.180338,0.088097,95,100
10,78.144126,0.095192,114,100
0,77.672623,0.075865,109,100


In [18]:
KNeighbors = df_KNeighbors.loc[18]
KNeighbors[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

scaler                           str
estimator       KNeighborsClassifier
imputer                SimpleImputer
sampler                          str
auc_mean                   76.285254
auc_confl                  76.250001
auc_confu                  76.320507
type2                             70
f1_mean                     0.767047
f1_confl                    0.732173
f1_confu                     0.80192
drop_columns                       0
pre_mean                    0.781214
rec_mean                    0.772996
Name: 18, dtype: object

In [19]:
# loading Random Forest Classifier
with open("outputs/output RandomForestClassifier.json", 'r') as f:
  data_Forest = json.load(f)

df_Forest = pd.json_normalize(data_Forest, record_path =['measurements'])
df_Forest["drop_columns"] = df_Forest["best_params.drop_columns__minimum_percentage_to_be_dropped"]
df_Forest["auc_std"] = df_Forest.apply(lambda row: np.std(row["auc"]), axis =1)
df_Forest["pre_mean"] = df_Forest.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
df_Forest["rec_mean"] = df_Forest.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
df_Forest["f1_std"] = df_Forest.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
df_Forest["f1_mean"] = df_Forest.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
df_Forest["type2"] = df_Forest.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
df_Forest["auc_confl"] = df_Forest["auc_mean"]- 1.96 * (df_Forest["auc_std"] / np.sqrt(10))
df_Forest["auc_confu"] = df_Forest["auc_mean"]+ 1.96 * (df_Forest["auc_std"] / np.sqrt(10))
df_Forest["f1_confl"] = df_Forest["f1_mean"]- 1.96 * (df_Forest["f1_std"] / np.sqrt(10))
df_Forest["f1_confu"] = df_Forest["f1_mean"]+ 1.96 * (df_Forest["f1_std"] / np.sqrt(10))
df_Forest.head()
df_Forest = df_Forest.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
df_Forest[["auc_mean", "auc_std", "type2", "drop_columns"]].head(20)

,auc_mean,auc_std,type2,drop_columns
15,78.332342,0.097413,79,100
12,78.205301,0.104771,79,100
3,78.190480,0.095344,78,100
9,78.160391,0.095387,77,100
16,78.105811,0.093092,96,75
1,78.103771,0.094111,96,75
0,78.065480,0.098726,78,100
4,78.041551,0.095939,99,75
13,77.903771,0.094006,98,75
10,77.889983,0.106215,92,75


In [20]:
Forest = df_Forest.loc[15]
Forest[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

scaler                  StandardScaler
estimator       RandomForestClassifier
imputer                  SimpleImputer
sampler                            str
auc_mean                     78.332342
auc_confl                    78.271965
auc_confu                    78.392719
type2                               79
f1_mean                       0.775015
f1_confl                      0.700043
f1_confu                      0.849987
drop_columns                       100
pre_mean                      0.815984
rec_mean                      0.789526
Name: 15, dtype: object

In [21]:
# loading Decision Tree Classifier
with open("outputs/output DecisionTrees.json", 'r') as f:
  data_Tree = json.load(f)

df_Tree = pd.json_normalize(data_Tree, record_path =['measurements'])
df_Tree["drop_columns"] = df_Tree["best_params.drop_columns__minimum_percentage_to_be_dropped"]
df_Tree["auc_std"] = df_Tree.apply(lambda row: np.std(row["auc"]), axis =1)
df_Tree["pre_mean"] = df_Tree.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
df_Tree["rec_mean"] = df_Tree.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
df_Tree["f1_std"] = df_Tree.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
df_Tree["f1_mean"] = df_Tree.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
df_Tree["type2"] = df_Tree.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
df_Tree["auc_confl"] = df_Tree["auc_mean"]- 1.96 * (df_Tree["auc_std"] / np.sqrt(10))
df_Tree["auc_confu"] = df_Tree["auc_mean"]+ 1.96 * (df_Tree["auc_std"] / np.sqrt(10))
df_Tree["f1_confl"] = df_Tree["f1_mean"]- 1.96 * (df_Tree["f1_std"] / np.sqrt(10))
df_Tree["f1_confu"] = df_Tree["f1_mean"]+ 1.96 * (df_Tree["f1_std"] / np.sqrt(10))
df_Tree.head()
df_Tree = df_Tree.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
df_Tree[["auc_mean", "auc_std", "type2", "drop_columns", 'scaler']].head(20)

,auc_mean,auc_std,type2,drop_columns,scaler
0,76.703708,0.059859,73,0,MaxAbsScaler
3,76.703708,0.059859,73,0,MinMaxScaler
7,76.703708,0.059859,73,0,PowerTransformer
10,76.703708,0.059859,73,0,RobustScaler
13,76.703708,0.059859,73,0,StandardScaler
16,76.703708,0.059859,73,0,str
6,75.907765,0.056369,81,0,Normalizer
2,74.731234,0.047840,105,0,MaxAbsScaler
5,74.731234,0.047840,105,0,MinMaxScaler
9,74.731234,0.047840,105,0,PowerTransformer


In [22]:
Tree = df_Tree.loc[16]

Tree[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

scaler                             str
estimator       DecisionTreeClassifier
imputer                  SimpleImputer
sampler                            str
auc_mean                     76.703708
auc_confl                    76.666607
auc_confu                     76.74081
type2                               73
f1_mean                       0.770171
f1_confl                      0.733067
f1_confu                      0.807274
drop_columns                         0
pre_mean                      0.785349
rec_mean                      0.776305
Name: 16, dtype: object

In [23]:
# Tree.to_excel(r'DTree.xlsx', index=False)

In [24]:
# loading SVC
with open("outputs/output SVC.json", 'r') as f:
  data_SVC = json.load(f)

df_SVC = pd.json_normalize(data_SVC, record_path =['measurements'])
df_SVC["drop_columns"] = df_SVC["best_params.drop_columns__minimum_percentage_to_be_dropped"]
df_SVC["auc_std"] = df_SVC.apply(lambda row: np.std(row["auc"]), axis =1)
df_SVC["pre_mean"] = df_SVC.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
df_SVC["rec_mean"] = df_SVC.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
df_SVC["f1_std"] = df_SVC.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
df_SVC["f1_mean"] = df_SVC.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
df_SVC["type2"] = df_SVC.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
df_SVC["auc_confl"] = df_SVC["auc_mean"]- 1.96 * (df_SVC["auc_std"] / np.sqrt(10))
df_SVC["auc_confu"] = df_SVC["auc_mean"]+ 1.96 * (df_SVC["auc_std"] / np.sqrt(10))
df_SVC["f1_confl"] = df_SVC["f1_mean"]- 1.96 * (df_SVC["f1_std"] / np.sqrt(10))
df_SVC["f1_confu"] = df_SVC["f1_mean"]+ 1.96 * (df_SVC["f1_std"] / np.sqrt(10))
df_SVC.head()
df_SVC = df_SVC.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
df_SVC[["auc_mean", "auc_std", "type2", "drop_columns"]].head(20)

,auc_mean,auc_std,type2,drop_columns
17,79.468591,0.065352,101.0,20.0
2,78.508412,0.076647,113.0,100.0
16,78.051257,0.084012,110.0,20.0
0,77.935901,0.073471,94.0,20.0
4,77.907379,0.083855,114.0,20.0
9,77.870632,0.084183,92.0,20.0
3,77.527738,0.073300,98.0,20.0
11,77.283387,0.075887,114.0,75.0
5,77.206434,0.071671,133.0,75.0
15,77.173656,0.078727,104.0,20.0


In [25]:
SVC = df_SVC.loc[9]
SVC[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

scaler          PowerTransformer
estimator                    SVC
imputer            SimpleImputer
sampler                      str
auc_mean               77.870632
auc_confl              77.818455
auc_confu              77.922809
type2                       92.0
f1_mean                 0.774087
f1_confl                0.716995
f1_confu                0.831179
drop_columns                20.0
pre_mean                0.800496
rec_mean                0.782909
Name: 9, dtype: object

In [26]:
# loading naïve bayes (bernoulli)
with open("outputs/output BernoulliNB.json", 'r') as f:
  data_bernoulli = json.load(f)

df_bernoulli = pd.json_normalize(data_bernoulli, record_path =['measurements'])
df_bernoulli["drop_columns"] = df_bernoulli["best_params.drop_columns__minimum_percentage_to_be_dropped"]
df_bernoulli["auc_std"] = df_bernoulli.apply(lambda row: np.std(row["auc"]), axis =1)
df_bernoulli["pre_mean"] = df_bernoulli.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
df_bernoulli["rec_mean"] = df_bernoulli.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
df_bernoulli["f1_std"] = df_bernoulli.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
df_bernoulli["f1_mean"] = df_bernoulli.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
df_bernoulli["type2"] = df_bernoulli.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
df_bernoulli["auc_confl"] = df_bernoulli["auc_mean"]- 1.96 * (df_bernoulli["auc_std"] / np.sqrt(10))
df_bernoulli["auc_confu"] = df_bernoulli["auc_mean"]+ 1.96 * (df_bernoulli["auc_std"] / np.sqrt(10))
df_bernoulli["f1_confl"] = df_bernoulli["f1_mean"]- 1.96 * (df_bernoulli["f1_std"] / np.sqrt(10))
df_bernoulli["f1_confu"] = df_bernoulli["f1_mean"]+ 1.96 * (df_bernoulli["f1_std"] / np.sqrt(10))
df_bernoulli.head()
df_bernoulli = df_bernoulli.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
df_bernoulli[["auc_mean", "auc_std", "type2", "drop_columns"]].head(20)

,auc_mean,auc_std,type2,drop_columns
15,79.068118,0.071319,111,8
16,78.913029,0.067901,115,8
17,78.758947,0.069558,119,8
9,77.518143,0.095385,109,35
10,77.389062,0.094922,114,35
11,77.384980,0.097225,114,35
12,76.922710,0.099803,116,60
0,76.840119,0.091046,102,20
6,76.840119,0.091046,102,20
18,76.840119,0.091046,102,20


In [27]:
bernoulli = df_bernoulli.loc[3]
bernoulli[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

scaler           MinMaxScaler
estimator         BernoulliNB
imputer         SimpleImputer
sampler                   str
auc_mean            76.230836
auc_confl           76.182989
auc_confu           76.278684
type2                     101
f1_mean              0.757561
f1_confl             0.704142
f1_confu              0.81098
drop_columns                8
pre_mean             0.779828
rec_mean              0.76623
Name: 3, dtype: object

In [28]:
# # loading naïve bayes (categorical)
# with open("outputs/output CategoricalNB.json", 'r') as f:
#   data_categorical = json.load(f)
#
# df_categorical = pd.json_normalize(data_categorical, record_path =['measurements'])
# df_categorical["drop_columns"] = df_XGBoost["best_params.drop_columns__minimum_percentage_to_be_dropped"]
# df_categorical["auc_std"] = df_categorical.apply(lambda row: np.std(row["auc"]), axis =1)
# df_categorical["pre_mean"] = df_categorical.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
# df_categorical["rec_mean"] = df_categorical.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
# df_categorical["f1_std"] = df_categorical.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
# df_categorical["f1_mean"] = df_categorical.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
# df_categorical["type2"] = df_categorical.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
# df_categorical["auc_confl"] = df_categorical["auc_mean"]- 1.96 * (df_categorical["auc_std"] / np.sqrt(10))
# df_categorical["auc_confu"] = df_categorical["auc_mean"]+ 1.96 * (df_categorical["auc_std"] / np.sqrt(10))
# df_categorical["f1_confl"] = df_categorical["f1_mean"]- 1.96 * (df_categorical["f1_std"] / np.sqrt(10))
# df_categorical["f1_confu"] = df_categorical["f1_mean"]+ 1.96 * (df_categorical["f1_std"] / np.sqrt(10))
# df_categorical.head()
# df_categorical = df_categorical.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
# df_categorical[["auc_mean", "auc_std", "type2", "drop_columns"]].head(7)

KeyError: 'auc'

In [ ]:
# categorical = df_categorical.loc[0]
# categorical[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

In [ ]:
# # loading naïve bayes (complement)
# with open("outputs/output CompleteNB.json", 'r') as f:
#   data_complement = json.load(f)
#
# df_complement = pd.json_normalize(data_complement, record_path =['measurements'])
# df_complement["drop_columns"] = df_complement["best_params.drop_columns__minimum_percentage_to_be_dropped"]
# df_complement["auc_std"] = df_complement.apply(lambda row: np.std(row["auc"]), axis =1)
# df_complement["pre_mean"] = df_complement.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
# df_complement["rec_mean"] = df_complement.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
# df_complement["f1_std"] = df_complement.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
# df_complement["f1_mean"] = df_complement.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
# df_complement["type2"] = df_complement.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
# df_complement["auc_confl"] = df_complement["auc_mean"]- 1.96 * (df_complement["auc_std"] / np.sqrt(10))
# df_complement["auc_confu"] = df_complement["auc_mean"]+ 1.96 * (df_complement["auc_std"] / np.sqrt(10))
# df_complement["f1_confl"] = df_complement["f1_mean"]- 1.96 * (df_complement["f1_std"] / np.sqrt(10))
# df_complement["f1_confu"] = df_complement["f1_mean"]+ 1.96 * (df_complement["f1_std"] / np.sqrt(10))
# df_complement.head()
# df_complement = df_complement.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
# df_complement[["auc_mean", "auc_std", "type2", "drop_columns"]].head(7)

In [ ]:
# complement = df_complement.loc[3]
# complement[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

In [ ]:
# # loading naïve bayes (gaussian)
# with open("outputs/output GaussianNB.json", 'r') as f:
#   data_gaussian = json.load(f)
#
# df_gaussian = pd.json_normalize(data_gaussian, record_path =['measurements'])
# df_gaussian["drop_columns"] = df_gaussian["best_params.drop_columns__minimum_percentage_to_be_dropped"]
# df_gaussian["auc_std"] = df_gaussian.apply(lambda row: np.std(row["auc"]), axis =1)
# df_gaussian["pre_mean"] = df_gaussian.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
# df_gaussian["rec_mean"] = df_gaussian.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
# df_gaussian["f1_std"] = df_gaussian.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
# df_gaussian["f1_mean"] = df_gaussian.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
# df_gaussian["type2"] = df_gaussian.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
# df_gaussian["auc_confl"] = df_gaussian["auc_mean"]- 1.96 * (df_gaussian["auc_std"] / np.sqrt(10))
# df_gaussian["auc_confu"] = df_gaussian["auc_mean"]+ 1.96 * (df_gaussian["auc_std"] / np.sqrt(10))
# df_gaussian["f1_confl"] = df_gaussian["f1_mean"]- 1.96 * (df_gaussian["f1_std"] / np.sqrt(10))
# df_gaussian["f1_confu"] = df_gaussian["f1_mean"]+ 1.96 * (df_gaussian["f1_std"] / np.sqrt(10))
# df_gaussian.head()
# df_gaussian = df_gaussian.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
# df_gaussian[["auc_mean", "auc_std", "type2", "drop_columns"]].head(20)

In [ ]:
# gaussian = df_gaussian.loc[20]
# gaussian[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

In [ ]:
# # loading naïve bayes (multinomial)
# with open("outputs/output MultinomialNB.json", 'r') as f:
#   data_multinomial = json.load(f)
#
# df_multinomial = pd.json_normalize(data_multinomial, record_path =['measurements'])
# df_multinomial["drop_columns"] = df_multinomial["best_params.drop_columns__minimum_percentage_to_be_dropped"]
# df_multinomial["auc_std"] = df_multinomial.apply(lambda row: np.std(row["auc"]), axis =1)
# df_multinomial["pre_mean"] = df_multinomial.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
# df_multinomial["rec_mean"] = df_multinomial.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
# df_multinomial["f1_std"] = df_multinomial.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
# df_multinomial["f1_mean"] = df_multinomial.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
# df_multinomial["type2"] = df_multinomial.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
# df_multinomial["auc_confl"] = df_multinomial["auc_mean"]- 1.96 * (df_multinomial["auc_std"] / np.sqrt(10))
# df_multinomial["auc_confu"] = df_multinomial["auc_mean"]+ 1.96 * (df_multinomial["auc_std"] / np.sqrt(10))
# df_multinomial["f1_confl"] = df_multinomial["f1_mean"]- 1.96 * (df_multinomial["f1_std"] / np.sqrt(10))
# df_multinomial["f1_confu"] = df_multinomial["f1_mean"]+ 1.96 * (df_multinomial["f1_std"] / np.sqrt(10))
# df_multinomial.head()
# df_multinomial = df_multinomial.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
# df_multinomial[["auc_mean", "auc_std", "type2", "drop_columns"]].head(5)

In [ ]:
# multinomial = df_multinomial.loc[3]
# multinomial[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

In [29]:
# loading logistic regression
with open("outputs/output LogisticRegression.json", 'r') as f:
  data_logistic = json.load(f)

df_logistic = pd.json_normalize(data_logistic, record_path =['measurements'])
df_logistic["drop_columns"] = df_logistic["best_params.drop_columns__minimum_percentage_to_be_dropped"]
df_logistic["auc_std"] = df_logistic.apply(lambda row: np.std(row["auc"]), axis =1)
df_logistic["pre_mean"] = df_logistic.apply(lambda row: np.mean(row["classification_report.weighted avg.precision"]), axis =1)
df_logistic["rec_mean"] = df_logistic.apply(lambda row: np.mean(row["classification_report.weighted avg.recall"]), axis =1)
df_logistic["f1_std"] = df_logistic.apply(lambda row: np.std(row["classification_report.weighted avg.f1-score"]), axis =1)
df_logistic["f1_mean"] = df_logistic.apply(lambda row: np.mean(row["classification_report.weighted avg.f1-score"]), axis =1)
df_logistic["type2"] = df_logistic.apply(lambda row: np.sum(row["confusion_matrix.(1, 0)"]), axis =1)
df_logistic["auc_confl"] = df_logistic["auc_mean"]- 1.96 * (df_logistic["auc_std"] / np.sqrt(10))
df_logistic["auc_confu"] = df_logistic["auc_mean"]+ 1.96 * (df_logistic["auc_std"] / np.sqrt(10))
df_logistic["f1_confl"] = df_logistic["f1_mean"]- 1.96 * (df_logistic["f1_std"] / np.sqrt(10))
df_logistic["f1_confu"] = df_logistic["f1_mean"]+ 1.96 * (df_logistic["f1_std"] / np.sqrt(10))
df_logistic.head()
df_logistic = df_logistic.sort_values(by = ["auc_mean", "auc_std", "type2", "drop_columns"], ascending = False)
df_logistic[["auc_mean", "auc_std", "type2", "drop_columns"]].head(20)

,auc_mean,auc_std,type2,drop_columns
17,78.818143,0.081598,111,20
14,78.633972,0.081839,114,20
2,78.319126,0.066004,116,20
3,77.893591,0.077889,103,20
16,77.696192,0.072278,116,20
13,77.665045,0.079287,120,20
19,77.646167,0.089447,114,20
15,77.641575,0.069453,103,20
12,77.563502,0.064914,105,35
5,77.563004,0.069132,121,20


In [30]:
df = pd.concat([df_Tree, df_XGBoost, df_SVC, df_logistic, df_Forest, df_bernoulli, df_KNeighbors])

In [33]:
df[["auc_mean", "auc_std", "type2", "drop_columns", "estimator"]].sort_values('type2')

,auc_mean,auc_std,type2,drop_columns,estimator
18,76.285254,0.056878,70.0,0.0,KNeighborsClassifier
0,76.703708,0.059859,73.0,0.0,DecisionTreeClassifier
3,76.703708,0.059859,73.0,0.0,DecisionTreeClassifier
7,76.703708,0.059859,73.0,0.0,DecisionTreeClassifier
10,76.703708,0.059859,73.0,0.0,DecisionTreeClassifier
...,...,...,...,...,...
8,76.439149,0.067503,138.0,20.0,SVC
7,75.542198,0.067355,147.0,20.0,SVC
12,NaN,NaN,NaN,NaN,SVC
13,NaN,NaN,NaN,NaN,SVC


In [35]:
df[["auc_mean", "auc_std",'f1_mean', "type2", "drop_columns", "estimator"]].sort_values('f1_mean', ascending=False)

,auc_mean,auc_std,f1_mean,type2,drop_columns,estimator
15,80.398731,0.097697,0.797751,83.0,100.0,KNeighborsClassifier
17,79.468591,0.065352,0.792116,101.0,20.0,SVC
17,79.798258,0.090229,0.791528,100.0,100.0,KNeighborsClassifier
15,79.068118,0.071319,0.787344,111.0,8.0,BernoulliNB
16,79.383997,0.092650,0.786896,109.0,100.0,KNeighborsClassifier
...,...,...,...,...,...,...
19,73.935316,0.051369,0.739903,103.0,0.0,DecisionTreeClassifier
8,73.366974,0.067874,0.734026,106.0,0.0,DecisionTreeClassifier
12,NaN,NaN,NaN,NaN,NaN,SVC
13,NaN,NaN,NaN,NaN,NaN,SVC


In [ ]:
Tree = df_Tree.loc[16]

Tree[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]

In [ ]:
logistic = df_logistic.loc[6]
logistic[["scaler", "estimator", "imputer", "sampler", "auc_mean", "auc_confl", "auc_confu", "type2", "f1_mean", "f1_confl", "f1_confu", "drop_columns", "pre_mean", "rec_mean"]]